In [ ]:
import random
import time
import numpy as np
import pickle
import os

import aiida
from aiida.engine import run
from aiida.orm import Dict, KpointsData, StructureData, load_code, load_group
aiida.load_profile()

In [ ]:
# Takes input_positions list and return symmetrized coordinates
# Symmetrize 3 points in cartesian to 3 coordinates: 
# distance of 2 points to the other (center) and angle between the distances
# where dh1>dh2
def symmetrize3p(inputpos):
    
    pH1, pH2, pO = np.array(inputpos) #position array with H1, H2, O order
    
    #redefine the atoms positions relative to O and the found plane to get 2 dimensional coordinates
    #we find the distance between H1, H2 and O to the relative coordinates through u and v
    u = pH1-pO #numpy array (vector) of H1 relative position to O
    v = pH2-pO #numpy array (vector) of H2 relative position to O
    udotv = np.dot(u, v) # dot product in numpy arrays
    #norm of u,v
    nu = np.linalg.norm(u)
    nv = np.linalg.norm(v)
    
    #check to swap vectors to define Mx and My
    if nu >= nv:
        Mx = u
        My = v
    else: 
        Mx = v
        My = u
        
    # Find the angle between Mx and My
    xdoty = np.dot(Mx,My)
    nudotnv = np.dot(nu, nv)
    theta = np.arccos(xdoty/nudotnv)
    
    #and then return the new symmetrized coordinates Mx,My,theta
    symm_coords = [np.linalg.norm(Mx), np.linalg.norm(My), theta]
    return np.array(symm_coords)

def desymmetrize3p(symm_coords):
    # Extract the symmetrized values
    dh1, dh2, theta = symm_coords #input in the form of symmetrize3p output

    # Assume the distance between H1 and O is dh1
    # Assume the distance between H2 and O is dh2
    # Calculate the coordinates of H1 and H2 relative to O
    h1x = dh1 * np.sin(theta / 2.0)
    h1y = dh1 * np.cos(theta / 2.0)
    h2x = dh2 * np.sin(theta / 2.0)
    h2y = dh2 * np.cos(theta / 2.0)
    ox = 5.0  # O is at the origin in the symmetrized coordinates
    oy = 5.0

    # Calculate the coordinates of H1 and H2 in Cartesian coordinates
    h1_x = ox + h1x
    h1_y = oy + h1y
    h2_x = ox - h2x
    h2_y = oy + h2y
    
    # Return the Cartesian coordinates as a list of arrays
    return [np.array([h1_x, h1_y, 5.0]), np.array([h2_x, h2_y, 5.0]), np.array([ox, oy, 5.0])]

In [ ]:
!verdi code list
#!verdi plugin list aiida.data | grep pseudo

In [ ]:
code = load_code('pw73@localhost')

output_parameters = []
input_positions = []
molecules_dataset = []
builder = code.get_builder()
dataset_name = 'dataset_randcart_3000_5a_12.pickle'
n_iterations = 3000

In [ ]:
#cell and structure construction
outer_alat = 10.0 #angstrom 
inner_alat = 5. #angstrom
cell = [[outer_alat, 0., 0.,],
        [0., outer_alat, 0.,],
        [0., 0., outer_alat,],
       ]
#Cartesian coordinates bounds
lower_bound = (outer_alat - inner_alat) / 2.
higher_bound = (outer_alat + inner_alat) / 2.

#Symmetric coordinates bounds
lower_Hbound = 0.6 #defines the lowest distance between atoms, set at ~0.1 A lower than O-H bond lenght
higher_Hbound = (higher_bound - lower_bound) / np.sqrt(2.)
H_halfdistance = 0.3
lower_theta = np.arcsin(H_halfdistance/(lower_Hbound))
hx=0.757497309 #Hydrogen x coordinate in Angstrom units
hy=0.5870459335442 #Hydrogen y coordinate in Angstrom units
print(higher_Hbound)
print(lower_theta)

In [ ]:
# Manual pseudos import
from aiida_pseudo.data.pseudo.upf import UpfData

upf_h = UpfData('H.upf')
upf_o = UpfData('O.upf')
builder.pseudos = {
    'O': upf_o,
    'H1': upf_h,
    'H2': upf_h,
}

In [ ]:
#build parameters for pw input file
parameters = Dict({
    'CONTROL': {
        'calculation': 'scf',
        'restart_mode': 'from_scratch',
        'wf_collect': True,
    },
    'SYSTEM': {

       'ecutwfc': 70.0, #using optimized cutoff for h20 molecule 70.0
        #'nat': 3,
        #'ntyp': 2,
        'nbnd': 8,
        'occupations': 'smearing',
        'degauss': 0.02,
    },
    'ELECTRONS': {
        'diagonalization':'cg',
        'mixing_beta': 0.2,  #changing these parameters won't change computation time by a lot,
        'conv_thr': 1.0e-6, #but increases/decreases convergence probability
    },
})

builder.parameters = parameters

In [ ]:
#Gamma kpoints
kpoints = KpointsData()
kpoints.set_kpoints_mesh([1, 1, 1])
builder.settings = Dict({'GAMMA_ONLY': True})
builder.kpoints = kpoints

# Run the calculation on [num_machines] CPU CORES, do not modify as 
# it runs on one pw.x compiled program, and it is compiled in multi core if on the same machine
# and kill it if it runs longer than 1800 seconds.
# Set ``withmpi`` to ``False`` if ``pw.x`` was compiled without MPI support.
builder.metadata.options = {
    'resources': {
        'num_machines': 1,
    },
    'max_wallclock_seconds': 1800,
    'withmpi': True,
}
#it is recommended to set daemon to 1 worker and modify the num_machines parameter to see how many cores are used

In [ ]:
start = time.time() 

for i in range(0,n_iterations):

    #Generating symmetric coordinates to cartesian ones
    rdH = random.uniform(0.5,1.5)
    symm_coords = np.array([random.uniform(lower_Hbound,higher_Hbound),#0.96,# 2.,# #  random.uniform(lower_Hbound,higher_Hbound),
                            random.uniform(lower_Hbound,higher_Hbound),#2.,#0.96,#random.uniform(2*lower_Hbound,higher_Hbound),
                            random.uniform(lower_theta,np.pi),#1.823869# #0.3721    #leaving one free parameter  1.823869 
                           ]) 
    cartesian_coords = desymmetrize3p(symm_coords)
        
    #random positions declaration based on symmetric generation
    H1_pos = cartesian_coords[0]
    H2_pos = cartesian_coords[1]
    O_pos = cartesian_coords[2]

    position_matrix = [H1_pos, H2_pos, O_pos] #position matrix with H1, H2, O order

    #building the structure with randomized coordinates in the cell
    structure = StructureData(cell=cell)
    structure.append_atom(position=H1_pos, symbols='H', name = 'H1') #Hydrogen atom h1
    structure.append_atom(position=H2_pos, symbols='H', name = 'H2') #Hydrogen atom h2
    structure.append_atom(position=O_pos, symbols='O', name = 'O') #Oxygen atom o

    #assign created structure to node builder
    builder.structure = structure

    #run calculation and store results and node
    results, node = run.get_node(builder)

    #building the molecule dataset in real-time
    if node.is_finished_ok:
        
        molecules_dataset.append( {
            'positions': symm_coords, #symmetrized cartesian positions
            'energy': results['output_parameters']['energy'] #energy of the system
        } )
        
    # Checking progress and time passed
    current = time.time()
    progress_percent = (i + 1) / n_iterations * 100
    time_elapsed = current - start
    # Elapsed time conversion
    elapsed_hours = int(time_elapsed // 3600)
    elapsed_minutes = int((time_elapsed % 3600) // 60)
    elapsed_seconds = int(time_elapsed % 60)
    
    mean_time_per_iteration = time_elapsed / (i + 1)
    estimated_remaining_time = mean_time_per_iteration * (n_iterations - (i+1))
    # Remaining Time conversion
    remaining_hours = int(estimated_remaining_time // 3600)
    remaining_minutes = int( (estimated_remaining_time % 3600) // 60 )
    remaining_seconds = int(estimated_remaining_time % 60)
    print(f"Iteration {i + 1} of {n_iterations} - Progression: {progress_percent:.2f}%")
    print(f'Time elapsed: {elapsed_hours} hours, {elapsed_minutes} minutes, {elapsed_seconds} seconds') 
    print(f'Estimated remaining time: {remaining_hours} hours, {remaining_minutes} minutes, {remaining_seconds} seconds')
    print(f'Successful calculation until now: {len(molecules_dataset)} out of {i+1}, { (len(molecules_dataset)/(i+1))*100:.1f}%')
end = time.time()

In [ ]:
print('total time elapsed for calculations =',end-start, 'seconds')
print('the cicle terminated with', len(molecules_dataset), 'successes out of', n_iterations, 'total calculations')
#print(molecules_dataset)

In [ ]:
#training dataset construction
#Save dataset
#create directory to save dataset
save_dir = os.path.join(os.getcwd(), 'saved_dataset')
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
dataset_path = os.path.join(save_dir, dataset_name)
with open(dataset_path, 'wb') as file:
    pickle.dump(molecules_dataset, file)
print('Saved dataset at %s ' % dataset_path , '\n')